# Part 04
----
앞부분의 전처리 된 파일들을 마지막으로 하나로 합쳐서 출력한다. 이 파트에서도 자잘한 추가적인 전처리가 이루어진다. 우선적으로 인구이동 데이터와 부동산 데이터 사이의 공통된 단위들로 묶는다. 종합 후의 파일 크기가 크지 않은 점과 추후 다른파트에서 분석의 용이성을 위해 아래의 두가지 방법으로 종합한다.

* 행정구역, 년
* 행정구역, 년, 월

종합하는 과정에서 우선적으로 전입사유 데이터가 카테고리인 점을 고려하여 새로 인코딩을 한다. 각 칼럼을 단순 총합을 할 경우, 대다수 칼럼이 전입세대의 수와 비례함으로 인한 칼럼간의 공선성의 문제가 있으므로 개별 칼럼마다 적절한 종합함수를 정의하여 사용한다.

In [1]:
%matplotlib inline
import re

In [2]:
# setting the folder path and read in data
path = 'raw_data\\'
df_pop = pd.read_csv(path + '03_pop.csv', encoding='utf-8')
df_estate = pd.read_csv(path + '03_estate.csv', encoding='utf-8')

In [3]:
# create new columns for price per area and log(price per area)
df_estate['ppa'] = df_estate.price / df_estate.area

In [4]:
df_estate['ppa_mean'] = np.nan
df_estate['ppa_median'] = np.nan
df_estate['trade_vol'] = 0

In [5]:
# custom aggregate functions the real estate data
def ppa_mean_agg(series):
    return df_estate.loc[series.index, 'ppa'].mean()

def ppa_median_agg(series):
    return df_estate.loc[series.index, 'ppa'].median()   

estate_agg_func_dic = {
    'built': 'mean',
    'ppa_mean': ppa_mean_agg,
    'ppa_median': ppa_median_agg,
    'trade_vol': 'count',
}

In [6]:
df_estate_agg_year = df_estate.groupby(['year', 'area_code']).agg(estate_agg_func_dic)
df_estate_agg_month = df_estate.groupby(['year', 'month', 'area_code']).agg(estate_agg_func_dic)

In [7]:
df_estate_agg_year['ppa_median_log'] = np.log(df_estate_agg_year.ppa_median)
df_estate_agg_month['ppa_median_log'] = np.log(df_estate_agg_month.ppa_median)

In [8]:
# just another sanity check sanity check
df_pop.loc[df_pop.reason == -1].head()

,year,month,area_code,move_in,foreign,reason,house_total,hh_head,head_age,head_male,mov_multi,mov_ppl,mov_male
423,2006,1,26260,1,1,-1,1,1,52.0,0,1,2,0
571,2006,1,27110,1,1,-1,2,2,36.0,0,0,2,0
962,2006,1,31140,1,0,-1,1,1,44.0,1,1,2,1
970,2006,1,31140,1,1,-1,2,0,-1.0,0,1,5,0
1323,2006,1,41310,1,1,-1,1,1,39.0,1,1,4,2


In [9]:
# remove entries with missing values
df_pop = df_pop.loc[df_pop.reason != -1]

In [10]:
# turning categorical values into binary vectors
df_r = pd.get_dummies(df_pop.reason, prefix='r')
df_pop = pd.concat([df_pop, df_r], axis=1).drop('reason', axis=1)

In [11]:
df_pop.head()

,year,month,area_code,move_in,foreign,house_total,hh_head,head_age,head_male,mov_multi,mov_ppl,mov_male,r_1,r_2,r_3,r_4,r_5,r_6,r_9
0,2006,1,11110,1,0,309,258,42.344961,178,121,588,326,1,0,0,0,0,0,0
1,2006,1,11110,1,0,32,8,48.500000,6,8,50,23,0,1,0,0,0,0,0
2,2006,1,11110,1,0,134,129,41.945736,87,59,264,135,0,0,1,0,0,0,0
3,2006,1,11110,1,0,12,11,38.000000,3,8,26,8,0,0,0,1,0,0,0
4,2006,1,11110,1,0,2,2,30.500000,1,0,2,1,0,0,0,0,1,0,0


In [12]:
# getting ready for aggregation
df_pop['mov_vol'] = np.nan
df_pop['mov_ppl_vol'] = np.nan
df_pop['pop_change_ratio'] = np.nan
df_pop['head_ratio'] = np.nan
df_pop['head_male_ratio'] = np.nan
df_pop['ppl_per_house'] = np.nan
df_pop['mov_male_ratio'] = np.nan
df_pop['mov_multi_ratio'] = np.nan
df_pop['sign'] = (-1) ** (df_pop.move_in + 1)

In [13]:
# custom aggregate functions for the population data
def mov_vol_agg(series):
    return df_pop.loc[series.index, 'house_total'].sum()

def mov_ppl_vol_agg(series):
    return df_pop.loc[series.index, 'mov_ppl'].sum()

def pop_change_ratio_agg(series):
    return ((df_pop.loc[series.index, 'mov_ppl'] * df_pop.loc[series.index, 'sign']).sum() /
             df_pop.loc[series.index, 'mov_ppl'].sum())

def head_ratio_agg(series):
    return df_pop.loc[series.index, 'hh_head'].sum() / df_pop.loc[series.index, 'house_total'].sum()

def head_male_ratio_agg(series):
    return df_pop.loc[series.index, 'head_male'].sum() / df_pop.loc[series.index, 'hh_head'].sum()

def ppl_per_house_agg(series):
    return df_pop.loc[series.index, 'mov_ppl'].sum() / df_pop.loc[series.index, 'house_total'].sum()

def mov_male_ratio_agg(series):
    return df_pop.loc[series.index, 'mov_male'].sum() / df_pop.loc[series.index, 'mov_ppl'].sum()

def head_age_agg(series):
    return ((df_pop.loc[series.index, 'head_age'] * df_pop.loc[series.index, 'hh_head']).sum() /
             df_pop.loc[series.index, 'hh_head'].sum())

def reason_ratio_agg(series):
    return ((series * df_pop.loc[series.index, 'house_total'] * df_pop.loc[series.index, 'sign']).sum() /
             df_pop.loc[series.index, 'house_total'].sum())

def mov_multi_ratio_agg(series):
    return df_pop.loc[series.index, 'mov_multi'].sum() / df_pop.loc[series.index, 'house_total'].sum()

pop_agg_func_dic = {
    'mov_vol': mov_vol_agg,
    'mov_ppl_vol': mov_ppl_vol_agg,
    'pop_change_ratio': pop_change_ratio_agg,
    'head_ratio': head_ratio_agg,
    'head_male_ratio': head_male_ratio_agg,
    'ppl_per_house': ppl_per_house_agg,
    'mov_male_ratio': mov_male_ratio_agg,
    'mov_multi_ratio': mov_multi_ratio_agg,
    'head_age': head_age_agg,
    'r_1': reason_ratio_agg,
    'r_2': reason_ratio_agg,
    'r_3': reason_ratio_agg,
    'r_4': reason_ratio_agg,
    'r_5': reason_ratio_agg,
    'r_6': reason_ratio_agg,
    'r_9': reason_ratio_agg,
}

In [14]:
# group things accordingly and aggregate the data using custom aggregate functions
df_pop_agg_year = df_pop.groupby(['year', 'area_code']).agg(pop_agg_func_dic)
df_pop_agg_month = df_pop.groupby(['year', 'month', 'area_code']).agg(pop_agg_func_dic)

In [15]:
# combine the population data and the real estate data into one
df_year = pd.concat([df_pop_agg_year, df_estate_agg_year], axis=1)
df_month = pd.concat([df_pop_agg_month, df_estate_agg_month], axis=1)

In [16]:
df_year.reset_index(inplace=True)
df_month.reset_index(inplace=True)

In [17]:
# see if there are any null values
df_year.loc[df_year.loc[:, 'built'].isnull()]

,year,area_code,mov_vol,mov_ppl_vol,pop_change_ratio,head_ratio,head_male_ratio,ppl_per_house,mov_male_ratio,mov_multi_ratio,...,r_3,r_4,r_5,r_6,r_9,built,ppa_mean,ppa_median,trade_vol,ppa_median_log
184,2006,46910,12878.0,17595.0,0.030520,0.543951,0.679515,1.366284,0.541290,0.202827,...,-0.024150,-0.007532,0.001165,0.001864,0.053191,NaN,NaN,NaN,NaN,NaN
412,2007,46910,12342.0,16734.0,-0.031313,0.528358,0.666769,1.355858,0.530417,0.197456,...,-0.019446,-0.004780,-0.000405,0.005186,0.052828,NaN,NaN,NaN,NaN,NaN
1324,2011,46910,10302.0,13821.0,-0.056798,0.563580,0.675680,1.341584,0.560958,0.188022,...,-0.052320,-0.016113,-0.000582,0.000777,-0.023491,NaN,NaN,NaN,NaN,NaN
1780,2013,46910,10370.0,13957.0,0.039192,0.565477,0.692019,1.345902,0.558573,0.193539,...,-0.032112,-0.011765,-0.003568,0.009450,0.005786,NaN,NaN,NaN,NaN,NaN
2008,2014,46910,10370.0,13803.0,-0.010215,0.555352,0.695086,1.331051,0.565747,0.188235,...,-0.048120,-0.014465,-0.001832,0.009547,0.004725,NaN,NaN,NaN,NaN,NaN
2259,2015,47940,3653.0,5061.0,-0.018376,0.693676,0.708761,1.385437,0.566884,0.215439,...,-0.048453,-0.010402,-0.003285,0.002737,-0.005475,NaN,NaN,NaN,NaN,NaN


In [18]:
# null built values above indicate there weren't any houses sold during the period
# since we cannot do an analysis on the housing prices over these data points
# we drop these data points
df_year = df_year.loc[~df_year.loc[:, 'built'].isnull()]
df_month = df_month.loc[~df_month.loc[:, 'built'].isnull()]

In [19]:
df_year.to_csv(path + '04_pop_estate_year.csv', index=False, encoding='utf-8')
df_month.to_csv(path + '04_pop_estate_month.csv', index=False, encoding='utf-8')